In [3]:
import pandas as pd
import numpy as np
import os

In [4]:
usecol_ = ["Chr", "Start", "End", "Ref", "Alt", "Func.refGene", "Otherinfo10", "Otherinfo11"]
dtype_ = {"Chr":"category",
          "Start":np.int32,
          "End":np.int32,
          "Ref":"category",
          "Alt":"category",
          "Func.refGene":"category",
          "Otherinfo10":"category",
          "Otherinfo11":"string"}

In [5]:
df = pd.read_csv("anno1.fixed.chrY.genes.VE_22251.hg38_multianno.csv", usecols=usecol_, dtype=dtype_)
df = pd.eval("DP = df.Otherinfo11.str.split(';',3).str[2].str.split('DP=').str[1].astype('int32')", target=df)
df = pd.eval("AF = df.Otherinfo11.str.split(';',5).str[4].str.split('AF=').str[1].astype('float')", target=df)

In [60]:
# this works on Chr, Ref, Alt, Func.refGene, Otherinfo10
def category_filters(**kwargs):
    new_df = df
    for key, value in kwargs.items():
        if key == "FuncrefGene":
            new_df = new_df[new_df["Func.refGene"] == value]
        else:
            new_df = new_df[new_df[key] == value]
    return new_df

In [7]:
# for Func.refGene, enter 'FuncrefGene' instead 
display(category_filters(FuncrefGene="exonic", Otherinfo10="PASS"))

,Chr,Start,End,Ref,Alt,Func.refGene,Otherinfo10,Otherinfo11,DP,AF
0,chrY,2787119,2787119,C,A,exonic,PASS,SAMPLE=VE_22251;TYPE=SNV;DP=415;VD=7;AF=0.0169...,415,0.0169
1,chrY,2787445,2787445,T,C,exonic,PASS,SAMPLE=VE_22251;TYPE=SNV;DP=317;VD=4;AF=0.0126...,317,0.0126
15,chrY,2865200,2865200,C,A,exonic,PASS,SAMPLE=VE_22251;TYPE=SNV;DP=173;VD=2;AF=0.0116...,173,0.0116
16,chrY,2866887,2866887,G,T,exonic,PASS,SAMPLE=VE_22251;TYPE=SNV;DP=232;VD=4;AF=0.0172...,232,0.0172
28,chrY,2961350,2961350,G,A,exonic,PASS,SAMPLE=VE_22251;TYPE=SNV;DP=152;VD=2;AF=0.0132...,152,0.0132
33,chrY,2978846,2978846,T,C,exonic,PASS,SAMPLE=VE_22251;TYPE=SNV;DP=182;VD=2;AF=0.011;...,182,0.0110
34,chrY,2979004,2979004,A,T,exonic,PASS,SAMPLE=VE_22251;TYPE=SNV;DP=148;VD=4;AF=0.027;...,148,0.0270
37,chrY,2979454,2979454,C,A,exonic,PASS,SAMPLE=VE_22251;TYPE=SNV;DP=232;VD=10;AF=0.043...,232,0.0431
43,chrY,5100327,5100327,G,T,exonic,PASS,SAMPLE=VE_22251;TYPE=SNV;DP=88;VD=88;AF=1;BIAS...,88,1.0000
44,chrY,5100614,5100614,T,G,exonic,PASS,SAMPLE=VE_22251;TYPE=SNV;DP=189;VD=189;AF=1;BI...,189,1.0000


In [8]:
# this assumes greater than i.e 'DP = 15' means keep DP vals > 15
def num_filter(**kwargs):
    new_df = df
    for key, value in kwargs.items():
        new_df.loc[:,key].where(new_df.loc[:,key] > value,inplace=True)
    return new_df.dropna()

In [9]:
display(num_filter(DP=150,AF=.15))

,Chr,Start,End,Ref,Alt,Func.refGene,Otherinfo10,Otherinfo11,DP,AF
44,chrY,5100614,5100614,T,G,exonic,PASS,SAMPLE=VE_22251;TYPE=SNV;DP=189;VD=189;AF=1;BI...,189.0,1.0


In [155]:
# subtraction algorithm
# first find all the same starts index in df1 and df2
def gen_unique_id(df1):    
    
    # ref alt ie ['A', 'C', 'T'] -> [65, 84, 49]
    def ascii_swap(col, cats):
        for cat in cats:
            cat_num = int(''.join([str(ord(x)) for x in cat]))
            col = np.where(col==cat, cat_num, col) 
        return pd.Series(col)
    
    ref_cats = np.array(pd.Categorical(df1["Ref"]).categories)
    alt_cats = np.array(pd.Categorical(df1["Alt"]).categories)
    
    num_ref = ascii_swap(np.array(df1["Ref"]), ref_cats).astype("string")
    num_alt = ascii_swap(np.array(df1["Ref"]), ref_cats).astype("string")
    
    num_start = df1["Start"].astype("string")
    num_end = df1["End"].astype("string")
    
    # add zeros in between columns?
    df1_len = len(num_start)
    num_zero = pd.Series(np.zeros(df1_len)).astype("string")
    
    # div col by num to make it small to avoid pd.to_numeric value error int out of range
    def d_c(col, num=10):
        return (pd.to_numeric(col)/num).to_string()

    num_uqid = pd.to_numeric(d_c(num_start) + d_c(num_end) + d_c(num_ref) + d_c(num_alt), downcast='float')
    #num_uqid = d_c(num_start) + d_c(num_end) + d_c(num_ref) + d_c(num_alt)
    
    # pd numeric was a fail, trying to make cols smaller is too difficult
    # switching to subtracting by aligning true false per num column for num(start end ref alt)
    
    return num_uqid

In [153]:
# if sub algorithm works, return df should have length = len(df)//2 - 3
display(gen_unique_id(df[["Start","End", "Ref", "Alt"]]))
# print(sub_v-p(df[len(df)//2 + 3:], df[:len(df)//2]))
# output is df1 w indices of where it is equal

ValueError: Unable to parse string "0       2787.119
1       2787.445
2       2787.569
3       2787.624
4       2841.892
5       2841.893
6       2842.127
7       2842.267
8       2845.560
9       2845.590
10      2848.335
11      2854.465
12      2854.469
13      2854.489
14      2854.800
15      2865.200
16      2866.887
17      2876.872
18      2881.949
19      2883.337
20      2883.454
21      2927.518
22      2930.670
23      2931.239
24      2938.908
25      2949.388
26      2953.898
27      2954.073
28      2961.350
29      2961.480
30      2964.922
31      2966.829
32      2977.012
33      2978.846
34      2979.004
35      2979.076
36      2979.347
37      2979.454
38      5028.609
39      5037.277
40      5056.989
41      5098.323
42      5099.519
43      5100.327
44      5100.614
45      5104.219
46      5104.298
47      5104.370
48      5104.431
49      5108.603
50      5114.387
51      5197.827
52      5199.349
53      5199.357
54      5199.365
55      5199.380
56      5199

In [154]:
print(np.array(pd.Categorical(df["Ref"]).categories))
# print(np.array(ref.to_numpy()).decode('ascii'))
# np.array(['a','b','c']).tostring().encode("ascii")
# print([''.join([ord(i) for i in x]) for x in list(ref)])

['-' 'A' 'AA' 'C' 'G' 'GCA' 'GGGC' 'T' 'TG' 'TT' 'TTCT']


In [ ]:
#TODO: make option to see what is NA and what is not NA
#TODO: make program and functions run on a class
# possible error of pd.to_numeric int out of range may come from long ref/alt strings